## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,52.92,88,90,3.15,overcast clouds
1,1,Mayumba,GA,-3.4320,10.6554,74.28,86,97,6.02,overcast clouds
2,2,Shingu,JP,33.7333,135.9833,79.97,93,100,15.30,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,40.66,70,75,23.02,broken clouds
4,4,Helena,US,46.5927,-112.0361,72.36,39,0,13.80,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,52.92,88,90,3.15,overcast clouds
1,1,Mayumba,GA,-3.4320,10.6554,74.28,86,97,6.02,overcast clouds
2,2,Shingu,JP,33.7333,135.9833,79.97,93,100,15.30,overcast clouds
4,4,Helena,US,46.5927,-112.0361,72.36,39,0,13.80,clear sky
5,5,Tateyama,JP,34.9833,139.8667,78.10,86,72,15.99,light rain
6,6,Faya,SA,18.3851,42.4509,64.92,44,98,4.61,overcast clouds
7,7,Kapaa,US,22.0752,-159.3190,84.52,67,0,9.22,clear sky
8,8,Sangmelima,CM,2.9333,11.9833,67.46,99,100,3.96,overcast clouds
10,10,Bathsheba,BB,13.2167,-59.5167,83.01,83,40,8.05,scattered clouds
11,11,Busselton,AU,-33.6500,115.3333,53.46,83,96,2.68,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,New Norfolk,AU,52.92,overcast clouds,-42.7826,147.0587,
1,Mayumba,GA,74.28,overcast clouds,-3.4320,10.6554,
2,Shingu,JP,79.97,overcast clouds,33.7333,135.9833,
4,Helena,US,72.36,clear sky,46.5927,-112.0361,
5,Tateyama,JP,78.10,light rain,34.9833,139.8667,
6,Faya,SA,64.92,overcast clouds,18.3851,42.4509,
7,Kapaa,US,84.52,clear sky,22.0752,-159.3190,
8,Sangmelima,CM,67.46,overcast clouds,2.9333,11.9833,
10,Bathsheba,BB,83.01,scattered clouds,13.2167,-59.5167,
11,Busselton,AU,53.46,overcast clouds,-33.6500,115.3333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
        
        hotel_df.loc[index, "Hotel Name"] = np.nan    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,New Norfolk,AU,52.92,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
1,Mayumba,GA,74.28,overcast clouds,-3.4320,10.6554,LIKWALE LODGE
2,Shingu,JP,79.97,overcast clouds,33.7333,135.9833,Shingu UI Hotel
4,Helena,US,72.36,clear sky,46.5927,-112.0361,"Holiday Inn Express & Suites Helena, an IHG Hotel"
5,Tateyama,JP,78.10,light rain,34.9833,139.8667,Tateyama Seaside Hotel
...,...,...,...,...,...,...,...
660,Nago,JP,82.83,overcast clouds,26.5881,127.9761,Hotel Yugaf Inn Okinawa
661,Sulangan,PH,83.12,overcast clouds,11.1373,123.7228,Maia's Beach Resort
662,Derzhavinsk,KZ,52.27,clear sky,51.1000,66.3167,Ooo.khim Baza
663,Drayton Valley,CA,57.43,broken clouds,53.2168,-114.9854,Service Plus Inns & Suites (Drayton Valley Hotel)


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,New Norfolk,AU,52.92,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
1,Mayumba,GA,74.28,overcast clouds,-3.4320,10.6554,LIKWALE LODGE
2,Shingu,JP,79.97,overcast clouds,33.7333,135.9833,Shingu UI Hotel
4,Helena,US,72.36,clear sky,46.5927,-112.0361,"Holiday Inn Express & Suites Helena, an IHG Hotel"
5,Tateyama,JP,78.10,light rain,34.9833,139.8667,Tateyama Seaside Hotel
...,...,...,...,...,...,...,...
660,Nago,JP,82.83,overcast clouds,26.5881,127.9761,Hotel Yugaf Inn Okinawa
661,Sulangan,PH,83.12,overcast clouds,11.1373,123.7228,Maia's Beach Resort
662,Derzhavinsk,KZ,52.27,clear sky,51.1000,66.3167,Ooo.khim Baza
663,Drayton Valley,CA,57.43,broken clouds,53.2168,-114.9854,Service Plus Inns & Suites (Drayton Valley Hotel)


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))